### 0 Helper function

In [1]:
import yaml

In [2]:
def load_file(filepath):
    with open(filepath,'r',encoding="utf-8") as f:
        return f.read()
def load_yaml(filepath):
    with open(filepath,'r',encoding="utf-8") as f:
        return yaml.safe_load(f)

<hr>

### 01 Input CVResume

In [3]:
resume_json = load_file("resume_json.txt")
print(resume_json)

{
  "contact_information": {
    "name": "Surya Teja Menta",
    "email": "-",
    "phone": "+91 8309584461",
    "linkedin": "-",
    "jobdb_link": "-",
    "portfolio_link": "suryatejamenta.co.in"
  },
  "professional_summary": {
    "has_summary": "Yes",
    "summary_points": [
      "I’m Surya Teja Menta, Results-driven Senior Data Scientist with 4+ years of experience in Data Science, Machine Learning (ML), and Generative AI (GenAI).",
      "Proven expertise in RAG (Retrieval-Augmented Generation), LLM fine-tuning, MLOps, and end-to-end AI solutions.",
      "Strong background in data analytics, statistical modeling, AI-powered automation, and scalable AI architectures.",
      "IBM Certified Professional Data Scientist with hands-on experience in LangChain, Hugging Face, OpenAI APIs, Vector Databases (ChromaDB, Pinecone), and cloud deployments (AWS, GCP).",
      "Passionate about AI research, model optimization, and developing cutting-edge AI solutions."
    ]
  },
  "education

<hr>

### 02 Promptbuilder

In [4]:
from google import genai
import json
import os
import yaml

In [5]:
class PromptBuilder:
    def __init__(self,section,criteria,cvresume):
        self.section  = section
        self.criteria = criteria[::-1]
        self.cvresume = cvresume
        with open("prompt.yaml","r",encoding="utf-8") as f:
            self.config = yaml.safe_load(f)
        # print(self.config)
    def build_response_template(self):
        return {
            "section": self.section,
            "scores": {
                c: {"score": 0, "feedback": ""} for c in self.criteria
            }
        }
    def build(self):
        config_role      = self.config['role']['role1']
        config_objective = self.config['objective']['objective1']
        config_section   = self.config['section']['section1']
        config_expected  = self.config['expected_content'][self.section]
        config_criteria  = ""
        for item in self.criteria:
            config_criteria = f"- {item}\n" + config_criteria
        config_scale     = self.config['scale']['score1']
        # config_output    = self.config['output']['format1']
        
        prompt_role      = f"Role :\n{config_role}"+"\n\n"
        prompt_objective = f"objectvie :\n{config_objective}"+"\n\n"
        prompt_section   = f"section :\n{config_section}"+"\n\n"
        prompt_expected  = f"expected :\n{config_expected}"+"\n"
        prompt_criteria  = f"Criteria :\n{config_criteria}"+"\n"
        prompt_scale     = f"Scale :\n{config_scale}"+"\n"
        prompt_output    = f"output :\n{json.dumps(self.build_response_template(), indent=2)}"+"\n\n"
        prompt_cvresume  = f"CV/Resume: \n{self.cvresume}"+"\n"

        prompt = prompt_role + prompt_objective + prompt_section \
            + prompt_expected + prompt_criteria + prompt_scale \
            + prompt_output + prompt_cvresume
        prompt = prompt.replace("<section_name>",self.section)
        return prompt

In [25]:
p1 = PromptBuilder( 
    section  = "Profile", 
    criteria = ["Completeness", "ContentQuality"],
    cvresume = resume_json
)
prompt1 = p1.build()
# print(prompt1)

In [26]:
p2 = PromptBuilder( 
    section  = "Summary", 
    criteria = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
    cvresume = resume_json
)
prompt2 = p2.build()
# print(prompt2)

In [27]:
p3 = PromptBuilder( 
    section  = "Education", 
    criteria = ["Completeness","RoleRelevance"],
    cvresume = resume_json
)
prompt3 = p3.build()
# print(prompt3)

In [28]:
p4 = PromptBuilder( 
    section  = "Experience", 
    criteria = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
    cvresume = resume_json
)
prompt4 = p4.build()
# print(prompt4)

In [29]:
p5 = PromptBuilder( 
    section  = "Activities", 
    criteria = ["Completeness", "ContentQuality","Grammar","Length"],
    cvresume = resume_json
)
prompt5 = p5.build()
# print(prompt5)

In [50]:
p6 = PromptBuilder( 
    section  = "Skills", 
    criteria = ["Completeness","Length","RoleRelevance"],
    cvresume = resume_json
)
prompt6 = p6.build()
# print(prompt6)

<hr>

### 03 LLMCaller

In [31]:
import re
import json

In [ ]:
class LlmCaller:
    def __init__(self,prompt):
        self.client = genai.Client(api_key="AIzaSy...")
        self.prompt = prompt
    def parser(self,resp):
        text = resp.text.strip()
        text = re.sub(r"^```json|```$", "", text).strip()
        data = json.loads(text)
        return data
    def hit(self):
        self.resp = self.client.models.generate_content(
            model    = "gemini-2.5-flash",
            contents = self.prompt
            )
        data = self.parser(self.resp)
        return data

In [ ]:
class LlmCaller:
    def __init__(self):
        self.client = genai.Client(api_key="AIza...")
    def parser(self,resp):
        text = resp.text.strip()
        text = re.sub(r"^```json|```$", "", text).strip()
        data = json.loads(text)
        return data
    def hit(self,prompt):
        self.prompt = prompt
        self.resp = self.client.models.generate_content(
            model    = "gemini-2.5-flash",
            contents = self.prompt
            )
        data = self.parser(self.resp)
        return data

In [55]:
op = LlmCaller()

In [66]:
op1 = op.hit(prompt1)
op2 = op.hit(prompt2)
op3 = op.hit(prompt3)
op4 = op.hit(prompt4)
op5 = op.hit(prompt5)
op6 = op.hit(prompt6)


In [58]:
op1

{'section': 'Profile',
 'scores': {'ContentQuality': {'score': 5,
   'feedback': "The content is exceptionally high quality. It clearly establishes the candidate's professional identity as a 'Results-driven Senior Data Scientist with 4+ years of experience'. Key specializations like RAG, LLM fine-tuning, MLOps, and Generative AI are prominently featured, along with relevant certifications and specific tools (LangChain, Hugging Face, AWS, GCP). The language is strong, impactful, and effectively positions the candidate as an expert in their field."},
  'Completeness': {'score': 5,
   'feedback': "The 'Professional Summary' serves as an excellent Profile section, comprehensively covering all expected elements. It clearly states the candidate's basic professional identity and precise positioning ('Senior Data Scientist'). It effectively functions as a career direction/headline and completely avoids unnecessary personal details, focusing solely on professional qualifications and aspirations

In [59]:
op2

{'section': 'Summary',
 'scores': {'RoleRelevance': {'score': 5,
   'feedback': "The summary is highly relevant, immediately identifying the candidate as a 'Results-driven Senior Data Scientist' and detailing expertise directly applicable to modern AI/ML roles, especially in Generative AI. It strongly positions the candidate for senior data science or AI engineering positions."},
  'Length': {'score': 2,
   'feedback': 'The summary is composed of 5 sentences (or bullet points that read as full sentences), which slightly exceeds the recommended 2-4 sentence length. While each sentence is impactful, condensing it slightly would improve conciseness.'},
  'Grammar': {'score': 5,
   'feedback': 'The grammar, spelling, and punctuation are excellent. The summary reads smoothly and professionally with no errors.'},
  'ContentQuality': {'score': 4,
   'feedback': "The content is strong, providing specific technical and domain strengths (RAG, LLM fine-tuning, MLOps, LangChain, AWS, GCP). It effe

In [60]:
op3

{'section': 'Education',
 'scores': {'RoleRelevance': {'score': 4,
   'feedback': "The Bachelor's Degree in Computer Science is highly relevant to a Data Scientist role, providing a strong foundational background. While the inclusion of HSC and SSC details is comprehensive in terms of academic history, these entries do not directly enhance the relevance to a professional data career beyond general education."},
  'Completeness': {'score': 5,
   'feedback': 'All expected information, including Institution name, Degree & field of study, Dates attended, GPA, and honors, is comprehensively provided for each educational entry, demonstrating excellent completeness.'}}}

In [61]:
op4

{'section': 'Experience',
 'scores': {'RoleRelevance': {'score': 5,
   'feedback': 'The experience section is exceptionally relevant to a Senior Data Scientist role, showcasing strong expertise in Machine Learning, Generative AI, Computer Vision (YOLOv8), NLP, RAG pipelines, LLM fine-tuning, and MLOps. The roles and responsibilities align perfectly with the stated career objective.'},
  'Length': {'score': 5,
   'feedback': 'The length of the experience section is optimal. Two roles, each with 5-6 clear bullet points, provide sufficient detail to showcase significant contributions without being overly verbose. This is well-suited for an experienced professional.'},
  'Grammar': {'score': 5,
   'feedback': 'The grammar, spelling, and punctuation throughout the experience section are excellent. The bullet points are clear, concise, and professionally written, demonstrating strong communication skills.'},
  'ContentQuality': {'score': 4,
   'feedback': "The content demonstrates strong tec

In [67]:
op5

{'section': 'Activities',
 'scores': {'Length': {'score': 0,
   'feedback': "The 'Activities' section is entirely missing from the resume. There is no dedicated section or relevant content provided."},
  'Grammar': {'score': 0,
   'feedback': "Since there is no 'Activities' section present in the resume, grammar cannot be evaluated for this section."},
  'ContentQuality': {'score': 0,
   'feedback': "There is no dedicated 'Activities' section. While 'Participated in paper presentations and won prizes' is mentioned under education, it is too brief and generic to be considered quality content for a dedicated activities section, and it does not meet the expectation of detailed project descriptions, competitions, or club activities with responsibilities and tools/tech."},
  'Completeness': {'score': 0,
   'feedback': "The 'Activities' section is completely missing. The resume does not include competitions, hackathons, club activities, or project descriptions outside of the professional exp

In [63]:
op6

{'section': 'Skills',
 'scores': {'RoleRelevance': {'score': 5,
   'feedback': 'The skills section is exceptionally relevant for a Senior Data Scientist role. It comprehensively covers essential areas like Data Science, Machine Learning, Deep Learning, NLP, Computer Vision, and Generative AI, including advanced topics like LLMs, RAG, and MLOps. Specific frameworks and tools such as TensorFlow, PyTorch, Langchain, Docker, and cloud platforms (GCP, AWS) are well-represented and directly align with the experience outlined, demonstrating a strong match for the target role.'},
  'Length': {'score': 4,
   'feedback': "The length is strong and appropriate for a Senior Data Scientist with 4+ years of experience. It provides a comprehensive list of technical and soft skills without being overly verbose, allowing a recruiter to quickly grasp the candidate's extensive capabilities across various domains."},
  'Completeness': {'score': 4,
   'feedback': "The section is very comprehensive in its te

<hr>

### 04 SectionScoreAggregator

In [68]:
import copy
class SectionScoreAggregator:
    def __init__(self,llm_output:dict):
        self.llm_output      = llm_output             # op
        self.config          = load_yaml("weight.yaml")         # config
        self.section         = llm_output["section"]  # Get section
        self.section_weights = self.config["weights"][self.section]  # config["weights"][section_key][criteria]
    def aggregate(self):
        ddict = {}
        total = 0.0
        # Protect multiple mutation when we run more than one time
        scores_copy = copy.deepcopy(self.llm_output["scores"])
        for criteria, body in scores_copy.items():
            raw = body["score"]
            w   = self.section_weights[criteria]
            weighted = raw / 5 * w
            body["score"] = weighted
            ddict[criteria] = body
            total = total + weighted
        return {
            "section": self.section,
            "total_score":total,
            "scores":ddict
        }

In [76]:
import copy
class SectionScoreAggregator:
    def __init__(self):
        self.config          = load_yaml("weight.yaml")         # config
    def aggregate(self,llm_output:dict):
        self.llm_output      = llm_output             # op
        self.section         = llm_output["section"]  # Get section
        self.section_weights = self.config["weights"][self.section]  # config["weights"][section_key][criteria]
        ddict = {}
        total = 0.0
        # Protect multiple mutation when we run more than one time
        scores_copy = copy.deepcopy(self.llm_output["scores"])
        for criteria, body in scores_copy.items():
            raw = body["score"]
            w   = self.section_weights[criteria]
            weighted = raw / 5 * w
            body["score"] = weighted
            ddict[criteria] = body
            total = total + weighted
        return {
            "section": self.section,
            "total_score":total,
            "scores":ddict
        }

In [77]:
agg = SectionScoreAggregator()

In [78]:
s1 = agg.aggregate(op1)
s1

{'section': 'Profile',
 'total_score': 20.0,
 'scores': {'ContentQuality': {'score': 10.0,
   'feedback': "The content is exceptionally high quality. It clearly establishes the candidate's professional identity as a 'Results-driven Senior Data Scientist with 4+ years of experience'. Key specializations like RAG, LLM fine-tuning, MLOps, and Generative AI are prominently featured, along with relevant certifications and specific tools (LangChain, Hugging Face, AWS, GCP). The language is strong, impactful, and effectively positions the candidate as an expert in their field."},
  'Completeness': {'score': 10.0,
   'feedback': "The 'Professional Summary' serves as an excellent Profile section, comprehensively covering all expected elements. It clearly states the candidate's basic professional identity and precise positioning ('Senior Data Scientist'). It effectively functions as a career direction/headline and completely avoids unnecessary personal details, focusing solely on professional qu

In [79]:
s2 = agg.aggregate(op2)
s2

{'section': 'Summary',
 'total_score': 40.0,
 'scores': {'RoleRelevance': {'score': 10.0,
   'feedback': "The summary is highly relevant, immediately identifying the candidate as a 'Results-driven Senior Data Scientist' and detailing expertise directly applicable to modern AI/ML roles, especially in Generative AI. It strongly positions the candidate for senior data science or AI engineering positions."},
  'Length': {'score': 4.0,
   'feedback': 'The summary is composed of 5 sentences (or bullet points that read as full sentences), which slightly exceeds the recommended 2-4 sentence length. While each sentence is impactful, condensing it slightly would improve conciseness.'},
  'Grammar': {'score': 10.0,
   'feedback': 'The grammar, spelling, and punctuation are excellent. The summary reads smoothly and professionally with no errors.'},
  'ContentQuality': {'score': 8.0,
   'feedback': "The content is strong, providing specific technical and domain strengths (RAG, LLM fine-tuning, MLOp

In [80]:
s3 = agg.aggregate(op3)
s3

{'section': 'Education',
 'total_score': 18.0,
 'scores': {'RoleRelevance': {'score': 8.0,
   'feedback': "The Bachelor's Degree in Computer Science is highly relevant to a Data Scientist role, providing a strong foundational background. While the inclusion of HSC and SSC details is comprehensive in terms of academic history, these entries do not directly enhance the relevance to a professional data career beyond general education."},
  'Completeness': {'score': 10.0,
   'feedback': 'All expected information, including Institution name, Degree & field of study, Dates attended, GPA, and honors, is comprehensively provided for each educational entry, demonstrating excellent completeness.'}}}

In [81]:
s4 = agg.aggregate(op4)
s4

{'section': 'Experience',
 'total_score': 46.0,
 'scores': {'RoleRelevance': {'score': 10.0,
   'feedback': 'The experience section is exceptionally relevant to a Senior Data Scientist role, showcasing strong expertise in Machine Learning, Generative AI, Computer Vision (YOLOv8), NLP, RAG pipelines, LLM fine-tuning, and MLOps. The roles and responsibilities align perfectly with the stated career objective.'},
  'Length': {'score': 10.0,
   'feedback': 'The length of the experience section is optimal. Two roles, each with 5-6 clear bullet points, provide sufficient detail to showcase significant contributions without being overly verbose. This is well-suited for an experienced professional.'},
  'Grammar': {'score': 10.0,
   'feedback': 'The grammar, spelling, and punctuation throughout the experience section are excellent. The bullet points are clear, concise, and professionally written, demonstrating strong communication skills.'},
  'ContentQuality': {'score': 8.0,
   'feedback': "Th

In [82]:
s5 = agg.aggregate(op5)
s5

{'section': 'Activities',
 'total_score': 0.0,
 'scores': {'Length': {'score': 0.0,
   'feedback': "The 'Activities' section is entirely missing from the resume. There is no dedicated section or relevant content provided."},
  'Grammar': {'score': 0.0,
   'feedback': "Since there is no 'Activities' section present in the resume, grammar cannot be evaluated for this section."},
  'ContentQuality': {'score': 0.0,
   'feedback': "There is no dedicated 'Activities' section. While 'Participated in paper presentations and won prizes' is mentioned under education, it is too brief and generic to be considered quality content for a dedicated activities section, and it does not meet the expectation of detailed project descriptions, competitions, or club activities with responsibilities and tools/tech."},
  'Completeness': {'score': 0.0,
   'feedback': "The 'Activities' section is completely missing. The resume does not include competitions, hackathons, club activities, or project descriptions ou

In [83]:
s6 = agg.aggregate(op6)
s6

{'section': 'Skills',
 'total_score': 26.0,
 'scores': {'RoleRelevance': {'score': 10.0,
   'feedback': 'The skills section is exceptionally relevant for a Senior Data Scientist role. It comprehensively covers essential areas like Data Science, Machine Learning, Deep Learning, NLP, Computer Vision, and Generative AI, including advanced topics like LLMs, RAG, and MLOps. Specific frameworks and tools such as TensorFlow, PyTorch, Langchain, Docker, and cloud platforms (GCP, AWS) are well-represented and directly align with the experience outlined, demonstrating a strong match for the target role.'},
  'Length': {'score': 8.0,
   'feedback': "The length is strong and appropriate for a Senior Data Scientist with 4+ years of experience. It provides a comprehensive list of technical and soft skills without being overly verbose, allowing a recruiter to quickly grasp the candidate's extensive capabilities across various domains."},
  'Completeness': {'score': 8.0,
   'feedback': "The section is

<hr>

### 05 GlobalAggregator

In [84]:
from datetime import datetime,timezone,timedelta

In [85]:
class GlobalAggregator:
    def __init__(self,SectionScoreAggregator_output:list):
        self.section_outputs = SectionScoreAggregator_output
        self.timestamp       = str(datetime.now(tz=(timezone(timedelta(hours=7)))))
        self.model_config    = load_yaml("model.yaml")     # should include model name
        self.weight_config   = load_yaml("weight.yaml")    # includes weights + version
        self.prompt_config   = load_yaml("prompt.yaml")    # includes prompt version
    def fn1(self):
        weights = self.weight_config["weights"]
        contribution = {}
        total = 0.0
        for section_data in self.section_outputs:
            section_name    = section_data["section"]
            total_score     = section_data["total_score"]
            section_weight  = weights[section_name]["section_weight"]
            section_contrib = total_score * section_weight
            contribution[section_name] = {
                "section_total": total_score,
                "section_weight": section_weight,
                "contribution": section_contrib
            }
            total = total + section_contrib
        return {
            "final_resume_score":total,
            "section_contribution":contribution
        }
    def fn2(self):
        details = {}
        for section_data in self.section_outputs:
            details[section_data["section"]] = {
                'total_score':section_data['total_score'],
                'scores':section_data['scores']
            }
        return details
    def fn3(self):
        return {
            "model_name": self.model_config['model']['generation_model'],
            "timestamp": self.timestamp,
            "weights_version": self.weight_config.get("version", "unknown"),
            "prompt_version": self.prompt_config.get("version", "unknown")
        }
    def fn0(self):
        conclution_part = self.fn1()
        detail_part     = self.fn2()
        metadata_part   = self.fn3()
        return {
            "conclution":conclution_part,
            "section_detail":detail_part,
            "metadata":metadata_part
        }

In [86]:
x = GlobalAggregator(
    SectionScoreAggregator_output = [s1,s2,s3,s4,s5,s6]
)

In [87]:
x.fn0()

{'conclution': {'final_resume_score': 24.0,
  'section_contribution': {'Profile': {'section_total': 20.0,
    'section_weight': 0.1,
    'contribution': 2.0},
   'Summary': {'section_total': 40.0,
    'section_weight': 0.1,
    'contribution': 4.0},
   'Education': {'section_total': 18.0,
    'section_weight': 0.2,
    'contribution': 3.6},
   'Experience': {'section_total': 46.0,
    'section_weight': 0.2,
    'contribution': 9.200000000000001},
   'Activities': {'section_total': 0.0,
    'section_weight': 0.2,
    'contribution': 0.0},
   'Skills': {'section_total': 26.0,
    'section_weight': 0.2,
    'contribution': 5.2}}},
 'section_detail': {'Profile': {'total_score': 20.0,
   'scores': {'ContentQuality': {'score': 10.0,
     'feedback': "The content is exceptionally high quality. It clearly establishes the candidate's professional identity as a 'Results-driven Senior Data Scientist with 4+ years of experience'. Key specializations like RAG, LLM fine-tuning, MLOps, and Generative

<hr>

In [94]:
class helperfunction():
    def load_file(self,filepath):
        with open(filepath,'r',encoding="utf-8") as f:
            return f.read()
    def load_yaml(self,filepath):
        with open(filepath,'r',encoding="utf-8") as f:
            return yaml.safe_load(f)

class A(helperfunction):
    def __init__(self):
        pass
    def fn(self):
        self.config = self.load_yaml("model.yaml")
        return self.config

a = A()
a.fn()

{'model': {'provider': 'google',
  'embedding_model': 'text-embedding-004',
  'generation_model': 'gemini-2.5-flash'}}